In [ ]:
import torch

In [ ]:
X = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
limit = int(input())

larger_than_limit_sum = (X[X > limit]).sum()

print(larger_than_limit_sum)

 8


tensor(9)


# Неделя 2. Строим первую нейронную сеть

2.6 Семинар: Реализация градиентного спуска (часть 1)

In [7]:
import torch

Допустим, у нас есть функция f(x)=loge(x +3)f(x) = log_{e}(x + 3)f(x)=loge​(x +3). Мы выбрали начальное приближение xt=0 =7x^{t=0} = 7xt=0 =7 . И шаг градиентного спуска α=10\alpha=10α=10

Чему будет равен xt=1x^{t=1}xt=1? 

In [6]:
x = torch.tensor(7.0, requires_grad=True)
f = torch.log(x + 3)
f.backward()
lr = 10
x.data = x.data - lr * x.grad.data
print(x.data)

tensor(6.)


Допустим, у нас есть функция f(X)=∑loge(xij+1)f(X) = \sum{log_{e}(x_{ij} + 1)}f(X)=∑loge​(xij​+1), где XXX - тензор размера 2x2. Мы выбрали начальное приближение Xt=0 =[[1,2],[4,5]]X^{t=0} = [[1, 2], [4, 5]]Xt=0 =[[1,2],[4,5]] . И шаг градиентного спуска α=10\alpha=10α=10

Чему будет равен Xt=1X^{t=1}Xt=1? 

In [11]:
X = torch.tensor([[1.0, 2.0], [4.0, 5.0]], requires_grad=True)
f = torch.log(X + 1).sum()
f.backward()
lr = 10
X.data = X.data - lr * X.grad.data
print(X.data)

tensor([[-4.0000, -1.3333],
        [ 2.0000,  3.3333]])


Реализуйте расчет градиента для функции f(w)=∏i,jloge(loge(wi,j +7))f(w) = \prod\limits_{i,j}{log_{e}(log_{e}({w_{i,j} + 7}}))f(w)=i,j∏​loge​(loge​(wi,j​ +7)) в точке w =[[5,10],[1,2]]w = [[5, 10], [1, 2]]w =[[5,10],[1,2]]

Подсказка: перемножить все значения функции можно с помощью метода .prod()

In [14]:
w = torch.tensor([[5.,10.],
                  [1.,2.]], requires_grad=True)

#######
device = torch.device('cuda:0' 
                      if torch.cuda.is_available() 
                      else 'cpu')
w = w.to(device)
#######

function = torch.log(torch.log(w + 7)).prod()
#function = (w+7).log().log().prod()

function.backward()

print(w.grad, '<- gradient')

tensor([[0.0201, 0.0109],
        [0.0449, 0.0351]]) <- gradient
